# Measuring the Entropy of English Language

In [29]:
import numpy             as np 
import matplotlib.pyplot as plt
import pandas            as pd
import glob
import re

In [16]:
##########################################################################
# Path where the books in TXT format are located
##########################################################################
path = 'Gutenberg/txt'

In [17]:
##########################################################################
# Getting file names (book titles)
##########################################################################
f_names = []
for file in glob.glob(path + "/*.txt"):
    f_names.append(file)

In [20]:
##########################################################################
# Let's print of to check
##########################################################################
print(f_names[0])

Gutenberg/txt\Abraham Lincoln___Lincoln Letters.txt


Let's first load "Robson Crusoe" by Daniel Defoe. But first we have to define soma auxiliary functions to pre-process the text file!

In [71]:
##########################################################################
# Define class bookwork, to read and prepare the text for us!
##########################################################################
class BookWorm():
    
    def __init__(self,):
        None

    ##########################################################################
    # This function you read the book file given its path
    ##########################################################################
    def read_book(self, path):
        with open(path, 'r') as file_object:
            book = file_object.read()
        return book

    ##########################################################################
    # This function receives the raw text and remove special characters
    ##########################################################################
    def remove_special(self, raw_text):
        final = [re.sub(r"[^a-zA-Z0-9]+", ' ', k) for k in raw_text.split("\n")]
        return final

    ##########################################################################
    # This function vectorizes the book , i.e. concatenates every sentence 
    # in a single list each sentence is separated by a space
    ##########################################################################
    def vectorize(self, text_no_special):
        text = ''
        for sentence in text_no_special:
            text = text + ' ' + sentence
        text = re.sub(' +', ' ', text) # Removes double spaces
        if text[0] == ' ':
            return text[1:].lower() # Rerurn only in lowercase
        else:
            return text.lower() # Rerurn only in lowercase

In [72]:
# Instatiate a BookWork object
bw = BookWorm()
# Reading "Robison Crusoe"
book     = bw.read_book('Gutenberg/txt/Daniel Defoe___The Life and Adventures of Robinson Crusoe (1808).txt')
# Removing special characters (Book with no special characters)
book_ns  = bw.remove_special(book)
# Vectorizing book
book_v   = bw.vectorize(book_ns)

Let's check the final result

In [75]:
print(book_v[:1000])

the life and adventures of robinson crusoe by daniel de foe london 18o8 illustration i had one labour to make me a canoe which at last i finished the life of de foe daniel de foe was descended from a respectable family in the county of northampton and born in london about the year 1663 his father james foe was a butcher in the parish of st giles s cripplegate and a protestant dissenter why the subject of this memoir prefixed the de to his family name cannot now be ascertained nor did he at any period of his life think it necessary to give his reasons to the public the political scribblers of the day however thought proper to remedy this lack of information and accused him of possessing so little of the amor patriae as to make the addition in order that he might not be taken for an englishman though this idea could have had no other foundation than the circumstance of his having in consequence of his zeal for king william attacked the prejudices of his countrymen in his true born englis